In [ ]:
from langchain_community.document_loaders.pdf import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.docstore.document import Document
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.storage import InMemoryStore, LocalFileStore
from langchain.storage._lc_store import create_kv_docstore
# from langchain.retrievers import ParentDocumentRetriever
import json
from pathlib import Path
from numpy.linalg import norm
from numpy import dot
import uuid

class DATABASE:
    def __init__(self, db_path, embedding, parent_path=None):
        self.db_path = db_path
        self.parent_path = parent_path
        self.db = Chroma(collection_name="split_parents", persist_directory=db_path, embedding_function=embedding)
        self.parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap= 20, separators=[" {'"])
        self.child_splitter = RecursiveCharacterTextSplitter(
            separators=["\n","\n\n" "\\n", "FF0E FE00.", "\\n\\n", '",', '........', '. '],
            chunk_size=120,
            chunk_overlap=20,
            length_function=len,
            is_separator_regex=False,
        )
        self.store = self.initial_store()
        self.retriever = self.initial_retriever()
        
    def initial_store(self, ):
        fs = LocalFileStore(self.parent_path)
        store = create_kv_docstore(fs)
        return store
    
    def initial_retriever(self, ):
        retriever = CustomParentDocumentRetriever(
            vectorstore=self.db,
            docstore=self.store,
            child_splitter=self.child_splitter,
            parent_splitter=self.parent_splitter
        )
        return retriever
    
    def insert_document(self, file_path, chunk_size=1000, chunk_overlap=0, jq_schema=None):
        file_format = file_path.split('.')[-1]
        if file_format == 'pdf':
            pdf_loader = UnstructuredPDFLoader(file_path)
            pdf_pages = pdf_loader.load_and_split()
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
            texts = text_splitter.split_documents(pdf_pages)
        elif file_format == 'json':
            documents = json.loads(Path(file_path).read_text())
            txts = [str(i) for i in list(documents[jq_schema])]
            texts = [Document(page_content=txt, metadata={"source": file_path}) for txt in txts]
        elif file_format == 'csv':
            loader = CSVLoader(file_path=file_path)
            texts = loader.load()
        elif file_format == 'txt':
            loader = TextLoader(file_path)
            documents = loader.load()
            text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
            texts = text_splitter.split_documents(documents)
        elif file_format == 'docx':
            loader = Docx2txtLoader(file_path)
            documents = loader.load()
            text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
            texts = text_splitter.split_documents(documents)
        else:
            print(f'This function does not support {file_format} file format.')
            return
        # self.db.add_documents(texts)
        self.retriever.add_documents(texts, ids=None)
    def insert_processed_texts(self, documents, chunk_size=1000, chunk_overlap=0):
        text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        texts = text_splitter.split_documents(documents)
        self.retriever.add_documents(texts, ids=None)
    # def insert_book(name, ID, kind_of_book, authors, publisher, year_of_publication, location, pdf_path):
    #     json = '''
    #             "Tên sách": {name},
    #             "Loại sách": "Giáo trình",
    #             "ID": "20134011",
    #             "Tác giả": "Trần Nhật Quang, Phạm Văn Khoa",
    #             "Nhà xuất bản": "Đại học Quốc gia Tp. Hồ Chí Minh",
    #             "Năm xuất bản": "2023",
    #             "Vị trí": "kệ số 2",
    #             "Nội dung đầu sách":"a",
    #         '''
    #     print(json.format(name))
    def insert_book(self, json_file: str, jq_schema, ids=None):
        documents = json.loads(Path(json_file).read_text())[jq_schema]
        if ids is None:
            doc_ids = [str(uuid.uuid4()) for _ in documents]
        else:
            if len(documents) != len(ids):
                raise ValueError(
                    "Got uneven list of documents and ids. "
                    "If `ids` is provided, should be same length as `documents`."
                )
            doc_ids = ids
        for idx in range(len(documents)):
            # Add to parents
            self.retriever.docstore.mset([(doc_ids[idx], Document(str(documents[idx])))])
            child_doc_file = documents[idx]['Nội dung đầu sách']
            pdf_loader = UnstructuredPDFLoader(child_doc_file)
            child_doc = pdf_loader.load()
            
            docs = []
            # for i, doc in enumerate(child_doc):
            _id = doc_ids[idx]
            sub_docs = self.child_splitter.split_documents(child_doc)
            for _ in sub_docs:
                print(_)
            if self.retriever.child_metadata_fields is not None:
                for _doc in sub_docs:
                    _doc.metadata = {
                        k: _doc.metadata[k] for k in self.retriever.child_metadata_fields
                    }
            for _doc in sub_docs:
                _doc.metadata[self.retriever.id_key] = _id
            docs.extend(sub_docs)
        self.retriever.vectorstore.add_documents(docs)

    def similarity_search(self, query, k=4):
        return self.db.similarity_search_with_score(query, k)

In [ ]:
import fitz

pdf_document = "books/pdf_folder/gt_lap_trinh_python_can_ban.pdf"
doc = fitz.open(pdf_document)

for page_num in range(doc.page_count):
    page = doc[page_num]
    text = page.get_text("text")
    print(text)

doc.close()

In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("/home/xuanai/html/Bot/Knowledge/Books/PDF/Nghệ thuật - Ẩm thực/skv117999_4988.pdf",extract_images=True)
child_doc = loader.load()

In [2]:
text = ''
for doc in child_doc:
    text += doc.page_content
text

'TU SACH\nDAI HOC\nSUPHAM\nNGUYEN THU TUAN\nPHUONG PHAP\nNGHIEN CUU KHOA HOC\nGIAO DUC MI THUAT\nSKY1\nSP\nNHA XUAT BAN DAI HOC SU PHAMNs18.T883\nNGUYEN THU TUAN\nPHUONG PHAP\nNGHIEN CUU KHOA HOC\nGIAO DUC Mi THUAT\n0117998\nNHA XUAT BAN DAI HOC SU PHAMMUC LUC\nLOI NOI DAU..\nMUC TIEU MON HOC..\nChuong 1. NHUNG VAN DE CHUNG VE NGHIEN CUU KHOA HQC\nA. MUC TIEU HQC TAP\nB. NOI DUNG.\n1.1. Mot so khai niem lien quan.\n12\n1.2. Muc tieu cua nghien cuu khoa hoc\n1.3. Dac diem cua nghien cuu khoa hoc\n13\n1.4. Chuc nang cua nghien cuu khoa hoc\n16\n1.5. Cac loai hinh nghien cuu khoa hoc\n17\n1.6. Phan loai cac de tai nghien cuu khoa hoc giao duc\n18\n1.7. Noi dung nghien cuu cua khoa hoc giao duc Mi thuat\n20\nC. TAI LIEU THAM KHAO HO TRQ HOC TAP\n23\nD.CAU HOI ON TAP VA THAO LUAN\n23\nChuong 2. PHUONG PHAP NGHIEN CUU KHOA HOC GIAO DUC\n24\nA. MUC TIEU HOC TAP\n24\n24\nB. NQI DUNG..\n2.1. Mot s6 khai niem lien quan\n24\n2.2. Cac phuong phap nghién cuu khoa hoc giao duc\n27\nC. TAI LIEU THAM 

In [3]:
data[2].page_content

'NHUNGMAU CHUYEN\nVE PHONG CACH TU DUY\nCUA HO CHI MINH\n6 Chi Minh co phong cach tu duy rieng, rat\ndoc dao,khoa hoc va hieu qua, voi nhung\ndac trung noi bat la:\nPhong cach tu duy khoa hoc, cach mang va hien\ndai: Xuat phat ti long yéu nuoc va muc dich tim duong\ncuu nuoc, trong qua trinh hoat dong ly luan va thuc\ntién phong phu, Ho Chi Minh da hinh thanh trong\nminh mot phong cach tu duy khoa hoc, cach mang va\nhién dai.\nPhong cach tu duy doc lap, tu chu, sang tao: D6 la\nphong cach tu duy khong giao dieu,rap khuon,khong\nvay muon nguyen xi cua nguoi khac, het suc tranh loi\ncu, duong mon, tu minh tim toi, suy nghi, truy den tan\ncung ban chat cua su vat, hién tuong de tim ra chan ly,\nphu hop voi nhu cau va dieu kien thuc tien.Ho Chi Minh\nda khong ngung lam giau tri tue cua minh bang von'

In [ ]:
import fitz

pdf_document = "books/pdf_folder/cac_phuong_phap_co_ban_trong_danh_gia_cam_quan_thuc_pham.pdf"
doc = fitz.open(pdf_document)

for page_num in range(doc.page_count):
    page = doc[page_num]
    text = page.get_text("text")
    print(text)

doc.close()

In [ ]:
import os
import fitz

folder_path = "books"

# Iterate through all files in the "books" folder
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)

    # Check if it's a PDF file
    if file_name.endswith(".pdf"):
        # Open the PDF document
        doc = fitz.open(file_path)

        # Iterate through each page in the document
        for page_num in range(doc.page_count):
            # Get the current page
            page = doc[page_num]

            # Extract text from the page
            text = page.get_text("text")

            # Print the text content of the page
            print(f"File: {file_name}, Page {page_num + 1}:\n{text}\n{'='*20}")

        # Close the PDF document
        doc.close()


In [27]:
import API_keys

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings(chunk_size=1)
import sys
sys.path.append("../")
from methods import *


# Load book_info database
RobotInfo =  DATABASE(db_path='robot_knowledge', embedding=embedding, parent_path="parents")

In [4]:
# file = '/home/xuanai/dang_fs/www/html/Knowledge/final_so_tay_thu_vien_nh23_24_8223.docx'
# RobotInfo.insert_document(file)
RobotInfo.insert_processed_texts(data)

In [5]:
RobotInfo.retriever.get_relevant_documents("mảu chuyển 'chuyến đi tạo mặt trận của Bác nằm ở cuốn sách nào?")

[Document(page_content='- Chuyen di “tao mat tran" cua Bac\n39\nHong Khanh\nII- Nhung mau chuyen ve phong cach lam viec\ncua H6 Chi Minh\n44\n- Bac ve Tan Trao\n48\nChu Van Tan\n-Nhung ngay dau gan Bac\n51\nHoang Huu Khang\n- Mai mai nho loi Bac day: "Khong lam quan cach mang"\n58\nHa Van Tinh\n- Hinh anh Bac Ho duoc ghi sau trong tri nho cua toi\n61\nNguyen Van Luu\n- C6 Bac trong tim\n63\nDam Quang Trung', metadata={'source': 'books/pdf_folder/skv125300_2727.pdf', 'file_path': 'books/pdf_folder/skv125300_2727.pdf', 'page': 5, 'total_pages': 10, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'PDFill: Free PDF Writer and Tools', 'creationDate': "D:20231206160941+07'00'", 'modDate': "D:20231212083357+07'00'", 'trapped': ''}),
 Document(page_content='MUC LUC\nTrang\nLoi Nha xuat ban\n1. Nhung mau chuyen ve phong cach tu duy cua\nH6 Chi Minh\nNhung loi Bac day\nPham Van Khoa\n-Nhung ngay doi co quan cung Bac\n12\nHoang Huu Khang\n

In [20]:
len(text)

4752

In [38]:
text = ''
for i in data:
    print(i.page_content)

NHIEUTAC GIA
Nhurng mau chuyen ve
PHONG CACH
HO CHI MINH
(Xuat ban lan thur ba)
SKV125300
DH
ST
NHA XUAT BAN CHINH TRI QUOC GIA SU THAT
059,70L/ 92
10576
NHIEU TAC GIA
Nhung mau chuyen ve
PHONG CACH
HO CHI MINH
(Xuat ban lan thur ba)
THU VEN TRUONG DHSPKT
SKV
125300
NHA XUAT BAN CHINH TRI QUOC GIA SU THAT
Ha Noi - 2019
NHUNGMAU CHUYEN
VE PHONG CACH TU DUY
CUA HO CHI MINH
6 Chi Minh co phong cach tu duy rieng, rat
doc dao,khoa hoc va hieu qua, voi nhung
dac trung noi bat la:
Phong cach tu duy khoa hoc, cach mang va hien
dai: Xuat phat ti long yéu nuoc va muc dich tim duong
cuu nuoc, trong qua trinh hoat dong ly luan va thuc
tién phong phu, Ho Chi Minh da hinh thanh trong
minh mot phong cach tu duy khoa hoc, cach mang va
hién dai.
Phong cach tu duy doc lap, tu chu, sang tao: D6 la
phong cach tu duy khong giao dieu,rap khuon,khong
vay muon nguyen xi cua nguoi khac, het suc tranh loi
cu, duong mon, tu minh tim toi, suy nghi, truy den tan
cung ban chat cua su vat, hién tuong de tim ra chan 

In [19]:
text

'NHIEUTAC GIA\nNhurng mau chuyen ve\nPHONG CACH\nHO CHI MINH\n(Xuat ban lan thur ba)\nSKV125300\nDH\nST\nNHA XUAT BAN CHINH TRI QUOC GIA SU THAT059,70L/ 92\n10576\nNHIEU TAC GIA\nNhung mau chuyen ve\nPHONG CACH\nHO CHI MINH\n(Xuat ban lan thur ba)\nTHU VEN TRUONG DHSPKT\nSKV\n125300\nNHA XUAT BAN CHINH TRI QUOC GIA SU THAT\nHa Noi - 2019NHUNGMAU CHUYEN\nVE PHONG CACH TU DUY\nCUA HO CHI MINH\n6 Chi Minh co phong cach tu duy rieng, rat\ndoc dao,khoa hoc va hieu qua, voi nhung\ndac trung noi bat la:\nPhong cach tu duy khoa hoc, cach mang va hien\ndai: Xuat phat ti long yéu nuoc va muc dich tim duong\ncuu nuoc, trong qua trinh hoat dong ly luan va thuc\ntién phong phu, Ho Chi Minh da hinh thanh trong\nminh mot phong cach tu duy khoa hoc, cach mang va\nhién dai.\nPhong cach tu duy doc lap, tu chu, sang tao: D6 la\nphong cach tu duy khong giao dieu,rap khuon,khong\nvay muon nguyen xi cua nguoi khac, het suc tranh loi\ncu, duong mon, tu minh tim toi, suy nghi, truy den tan\ncung ban chat cua 

In [8]:
from langchain_openai import ChatOpenAI
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import RetrievalQA


retriever = RobotInfo.db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5, "k": 4}
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever, return_source_documents=True, verbose=True, input_key="question")

In [43]:
chain.invoke("tác giả của một số mẩu chủyện về đời sống hằng ngày của Bác")



> Entering new RetrievalQA chain...

> Finished chain.


{'question': 'tác giả của một số mẩu chủyện về đời sống hằng ngày của Bác',
 'result': 'Tác giả của một số mẩu chuyện về đời sống hằng ngày của Bác là Le Minh Hien.',
 'source_documents': [Document(page_content='Pham Van Khoa\n-Nhung ngay doi co quan cung Bac\n12\nHoang Huu Khang\n- Nguoi day chung toi cong tac van dong quan chung\n18', metadata={'author': '', 'creationDate': "D:20231206160941+07'00'", 'creator': '', 'doc_id': 'dbbf91ea-2e17-4f79-b210-7e8179900142', 'file_path': 'books/pdf_folder/skv125300_2727.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20231212083357+07'00'", 'page': 4, 'producer': 'PDFill: Free PDF Writer and Tools', 'source': 'books/pdf_folder/skv125300_2727.pdf', 'subject': '', 'title': '', 'total_pages': 10, 'trapped': ''}),
  Document(page_content='51\nHoang Huu Khang\n- Mai mai nho loi Bac day: "Khong lam quan cach mang"\n58\nHa Van Tinh', metadata={'author': '', 'creationDate': "D:20231206160941+07'00'", 'creator': '', 'doc_id': 'fe00da79-cd80-4e1

In [44]:
chain.invoke('tác giả của mot so mau chuyen ve doi song hang ngay cua Bac')



> Entering new RetrievalQA chain...

> Finished chain.


{'question': 'tác giả của mot so mau chuyen ve doi song hang ngay cua Bac',
 'result': 'Tác giả của một số mẩu chuyện về đời sống hàng ngày của Bác là Le Minh Hien.',
 'source_documents': [Document(page_content='186\nLe Minh Hien\n- Mot so mau chuyen ve doi song hang ngay cua Bac Ho\n190\nViet Phuong\n234', metadata={'author': '', 'creationDate': "D:20231206160941+07'00'", 'creator': '', 'doc_id': '8d0d0f74-4292-4c75-91c6-a27019414252', 'file_path': 'books/pdf_folder/skv125300_2727.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20231212083357+07'00'", 'page': 7, 'producer': 'PDFill: Free PDF Writer and Tools', 'source': 'books/pdf_folder/skv125300_2727.pdf', 'subject': '', 'title': '', 'total_pages': 10, 'trapped': ''}),
  Document(page_content='- Chuyen di “tao mat tran" cua Bac\n39\nHong Khanh\nII- Nhung mau chuyen ve phong cach lam viec\ncua H6 Chi Minh\n44', metadata={'author': '', 'creationDate': "D:20231206160941+07'00'", 'creator': '', 'doc_id': 'fe00da79-cd80-4e13-8de

In [21]:
from langchain.tools.retriever import create_retriever_tool
retriever = RobotInfo.db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.2, "k": 4}
)

tool = create_retriever_tool(
    retriever,
    "book_researcher",
    "tìm kiếm và trả lời các thông tin về sách và về thư viện cho người dùng",
)

In [21]:
tool.run({'messages': ['hãy nêu nội quy trong thư viện cho tôi']})

NameError: name 'tool' is not defined

#MULTIQUERY

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "chuyến đi tạo mặt trận của Bác nằm ở trang mấy"

llm = ChatOpenAI(temperature=0)

# we instantiated the retreiever above
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=RobotInfo.db.as_retriever(), llm=llm
)

# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Bác đã tạo mặt trận ở trang nào trong chuyến đi của mình?', '2. Trong chuyến đi của Bác, mặt trận được tạo ở trang số mấy?', '3. Ở trang nào trong chuyến đi, Bác đã tạo mặt trận?']


In [40]:
from typing import List
from langchain import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser


# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Bạn là một người trợ lí thông minh trong thư viện, có thể giải đáp các thắc mắc về sách cho sinh viên. 
    Những cuốn sách trong thư viện hầu hết là viết bằng tiếng việt và có thể không có dấu. 
    .,
    Original question: {question}""",
)

llm = ChatOpenAI(temperature=0)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

# Run
retriever = MultiQueryRetriever(
    retriever=RobotInfo.db.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.get_relevant_documents(
    query="tác giả của chuyến đi tạo mặt trận của Bác"
)

len(unique_docs)

OutputParserException: Invalid json output: Xin lỗi, tôi không thể trả lời câu hỏi này vì không có đủ thông tin về cuốn sách "Chuyến đi tạo mặt trận của Bác". Bạn có thể cung cấp thêm thông tin như tác giả, năm xuất bản hoặc nhà xuất bản để tôi có thể tìm kiếm thông tin chính xác hơn không?

In [7]:
import json



def import_author(book_title,authors):
    with open("books.json", "r") as read_file:
        data = json.load(read_file)
    for book in data["sách"]:
        if (book["tên sách"] == book_title):
            book["tác giả"] = authors
    # print(data)
    with open("books.json", "w", encoding="utf-8") as output_file:
        json.dump(data, output_file, indent=4, ensure_ascii=False)


In [8]:
def name_book(stt):
    if (stt == 0) :
        return 'giáo trình cơ sở công nghệ chế tạo máy'
    if (stt == 1):
        return 'giáo trình lập trình python căn bản'
    return 'các phương pháp cơ bản trong đánh giá cảm quan thực phẩm'
    

In [13]:
import os
def find_author_in_text(text):
    text = text.lower()
    line_ts = [line.strip() for line in text.split('\n') if line.strip().startswith('ts')]
    line_ths = [line.strip() for line in text.split('\n') if line.strip().startswith('ths')]
    authors = (line_ts+line_ths)
    return authors
def extract_author_data():
    i = 0
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
    # Check if it's a PDF file
        if file_name.endswith(".pdf"):
            # Open the PDF document
            doc = fitz.open(file_path)

            # Iterate through each page in the document
                # Get the current page
            for page_num in range(2):
                page = doc[page_num]

                    # Extract text from the page
                text = page.get_text("text")
                # print(text)

                    # Print the text content of the page
                print(f"File: {file_name}, Page {page_num + 1}:\\n{'='*20}")

                authors = find_author_in_text(text)
                if authors is not None:
                    print("hello")
                    import_author(name_book(i),authors)
            # Close the PDF document
            doc.close()
            i = i + 1
extract_author_data()

File: cac_phuong_phap_co_ban_trong_danh_gia_cam_quan_thuc_pham.pdf, Page 1:\n====================
hello
{'sách': [{'tên sách': 'giáo trình cơ sở công nghệ chế tạo máy', 'tác giả': [], 'lời mở đầu': 'xin chào', 'mục lục ': [{'id': 'string', 'firstname': 'dang', 'surname': 'ho van', 'motivation': 'string', 'share': 'string'}]}, {'tên sách': 'giáo trình lập trình python căn bản', 'tác giả': 'nobel', 'lời mở đầu': 'xin chào', 'laureates': [{'id': 'string', 'firstname': 'nguyen', 'surname': 'thanh', 'motivation': 'string', 'share': 'string'}]}, {'tên sách': 'các phương pháp cơ bản trong đánh giá cảm quan thực phẩm', 'tác giả': 'nobel', 'lời mở đầu': 'xin chào', 'laureates': [{'id': 'string', 'firstname': 'nguyen', 'surname': 'thanh', 'motivation': 'string', 'share': 'string'}]}]}
File: cac_phuong_phap_co_ban_trong_danh_gia_cam_quan_thuc_pham.pdf, Page 2:\n====================
hello
{'sách': [{'tên sách': 'giáo trình cơ sở công nghệ chế tạo máy', 'tác giả': ['ts. phạm thị hoàn'], 'lời mở đầu

In [14]:
with open("books.json", "r") as read_file:
        data = json.load(read_file)
print(data)

{'sách': [{'tên sách': 'giáo trình cơ sở công nghệ chế tạo máy', 'tác giả': ['ts. phạm thị hoàn'], 'lời mở đầu': 'xin chào', 'mục lục ': [{'id': 'string', 'firstname': 'dang', 'surname': 'ho van', 'motivation': 'string', 'share': 'string'}]}, {'tên sách': 'giáo trình lập trình python căn bản', 'tác giả': [], 'lời mở đầu': 'xin chào', 'laureates': [{'id': 'string', 'firstname': 'nguyen', 'surname': 'thanh', 'motivation': 'string', 'share': 'string'}]}, {'tên sách': 'các phương pháp cơ bản trong đánh giá cảm quan thực phẩm', 'tác giả': ['ts. trần nhật quang, ts. phạm văn khoa'], 'lời mở đầu': 'xin chào', 'laureates': [{'id': 'string', 'firstname': 'nguyen', 'surname': 'thanh', 'motivation': 'string', 'share': 'string'}]}]}
